In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset